# Simple Spiking Neural Network (SNN)

The aim of this project is to create a very simple spiking neural network, with the focus being on acheiving some specific behaviors:

* Graded potentials
	The neuron be able to receive small inputs that change its membrane potential
* Action potentials
	When the membrane potential reaches the AP Treshold, it should generate a spike that is passed along to other neurons
* Refractory period
	After generating a spike, the membrane potential should return to its resting potential and should be more resistant to input potentials for some amount of time
* Neuron fatigue
	A strong enough input to the neuron should generate several action potentials followed by a longer, and more resiliant, refractory period

# Library imports

In [5]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np